In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Edema"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
163: 1690
85: 1690
437: 1690
726: 1690
90: 1690
92: 1690
246: 1690
24: 1690
109: 1690
1690: 1690
26: 1690
7: 1690
41: 1690
3: 1690
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
1670/1690 [============================>.] - ETA: 0s - loss: 5.1033 - leftLayer1_loss: 0.1015 - midLayer1_loss: 1.4845 - rightLayer1_loss: 1.0736 - leftLayer2_loss: 0.0816 - midLayer2_loss: 1.4160 - rightLayer2_loss: 0.9461
Epoch 00006: val_loss improved from 5.34128 to 5.26174, saving model to ./experiments/0.009_weights.h5
1690/1690 [==============================] - 4s 3ms/step - loss: 5.1047 - leftLayer1_loss: 0.1014 - midLayer1_loss: 1.4848 - rightLayer1_loss: 1.0735 - leftLayer2_loss: 0.0816 - midLayer2_loss: 1.4169 - rightLayer2_loss: 0.9465 - val_loss: 5.2617 - val_leftLayer1_loss: 0.1003 - val_midLayer1_loss: 1.4800 - val_rightLayer1_loss: 1.0797 - val_leftLayer2_loss: 0.0927 - val_midLayer2_loss: 1.3758 - val_rightLayer2_loss: 1.1334
Epoch 7/11
1676/1690 [============================>.] - ETA: 0s - loss: 5.0422 - leftLayer1_loss: 0.0980 - midLayer1_loss: 1.4847 - rightLayer1_loss: 1.0343 - leftLayer2_loss: 0.0767 - midLayer2_loss: 1.4161 - rightLayer2_loss: 0.9325


  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 1690 steps, validate for 200 steps
Epoch 1/11
1669/1690 [============================>.] - ETA: 0s - loss: 6.5686 - leftLayer1_loss: 0.1228 - midLayer1_loss: 1.4159 - rightLayer1_loss: 1.7682 - leftLayer2_loss: 0.1135 - midLayer2_loss: 1.5615 - rightLayer2_loss: 1.5867
Epoch 00001: val_loss improved from inf to 6.28310, saving model to ./experiments/0.009999999999999998_weights.h5
1690/1690 [==============================] - 5s 3ms/step - loss: 6.5649 - leftLayer1_loss: 0.1228 - midLayer1_loss: 1.4163 - rightLayer1_loss: 1.7668 - leftLayer2_loss: 0.1134 - midLayer2_loss: 1.5618 - rightLayer2_loss: 1.5839 - val_loss: 6.2831 - val_leftLayer1_loss: 0.1208 - val_midLayer1_loss: 1.4206 - val_rightLayer1_loss: 1.6538 - val_leftLayer2_loss: 0.1120 - val_midLayer2_loss: 1.4577 - val_rightLayer2_loss: 1.5

1670/1690 [============================>.] - ETA: 0s - loss: 5.0032 - leftLayer1_loss: 0.0893 - midLayer1_loss: 1.4171 - rightLayer1_loss: 0.9776 - leftLayer2_loss: 0.0599 - midLayer2_loss: 1.5565 - rightLayer2_loss: 0.9028
Epoch 00011: val_loss improved from 5.12147 to 5.09613, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Edema: 0.49294626500671856

auprc 0Edema: 0.01701201645737696

auroc 1Edema: 0.5112538568283731

auprc 1Edema: 0.018300900585080663

auroc 2Edema: 0.6001533384794364

auprc 2Edema: 0.026130622213392213

auroc 3Edema: 0.6005468834187719

auprc 3Edema: 0.022701135639878398

auroc 4Edema: 0.595984829991665

auprc 4Edema: 0.03641781288525901

auroc 5Edema: 0.5446592136138619

auprc 5Edema: 0.01947052232657804

mean auroc: 0.5575907312231378

mean auprc: 0.023338835017927547

max auroc: 0.6005468834187719

max auprc: 0.03641781

Epoch 4/11
1689/1690 [============================>.] - ETA: 0s - loss: 5.3784 - leftLayer1_loss: 0.1055 - midLayer1_loss: 1.4616 - rightLayer1_loss: 1.2592 - leftLayer2_loss: 0.0924 - midLayer2_loss: 1.4630 - rightLayer2_loss: 0.9967
Epoch 00004: val_loss improved from 5.66350 to 5.49994, saving model to ./experiments/0.010999999999999998_weights.h5
1690/1690 [==============================] - 4s 3ms/step - loss: 5.3784 - leftLayer1_loss: 0.1055 - midLayer1_loss: 1.4616 - rightLayer1_loss: 1.2592 - leftLayer2_loss: 0.0924 - midLayer2_loss: 1.4629 - rightLayer2_loss: 0.9967 - val_loss: 5.4999 - val_leftLayer1_loss: 0.1041 - val_midLayer1_loss: 1.4625 - val_rightLayer1_loss: 1.2313 - val_leftLayer2_loss: 0.0993 - val_midLayer2_loss: 1.4281 - val_rightLayer2_loss: 1.1747
Epoch 5/11
1685/1690 [============================>.] - ETA: 0s - loss: 5.2502 - leftLayer1_loss: 0.1022 - midLayer1_loss: 1.4608 - rightLayer1_loss: 1.1751 - leftLayer2_loss: 0.0852 - midLayer2_loss: 1.4646 - rightLayer

...

1687/1690 [============================>.] - ETA: 0s - loss: 4.9454 - leftLayer1_loss: 0.0801 - midLayer1_loss: 1.3696 - rightLayer1_loss: 0.9399 - leftLayer2_loss: 0.0595 - midLayer2_loss: 1.5918 - rightLayer2_loss: 0.9045
Epoch 00011: val_loss improved from 5.05736 to 5.03605, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Edema: 0.7232938139001964

auprc 0Edema: 0.03687890591598152

auroc 1Edema: 0.3461155168204999

auprc 1Edema: 0.012741516791752045

auroc 2Edema: 0.8162035173834925

auprc 2Edema: 0.10757848228863637

auroc 3Edema: 0.47040314440097375

auprc 3Edema: 0.01693847476697128

auroc 4Edema: 0.4262724509745708

auprc 4Edema: 0.014726153021222176

auroc 5Edema: 0.5944937246131077

auprc 5Edema: 0.028437489672840627

mean auroc: 0.5627970280154735

mean auprc: 0.036216837076234

max auroc: 0.8162035173834925

max auprc: 0.1075784822

Epoch 4/11
1676/1690 [============================>.] - ETA: 0s - loss: 5.3070 - leftLayer1_loss: 0.1026 - midLayer1_loss: 1.4926 - rightLayer1_loss: 1.1750 - leftLayer2_loss: 0.0963 - midLayer2_loss: 1.4642 - rightLayer2_loss: 0.9764
Epoch 00004: val_loss improved from 5.48434 to 5.33217, saving model to ./experiments/0.012999999999999996_weights.h5
1690/1690 [==============================] - 4s 3ms/step - loss: 5.3068 - leftLayer1_loss: 0.1026 - midLayer1_loss: 1.4928 - rightLayer1_loss: 1.1746 - leftLayer2_loss: 0.0963 - midLayer2_loss: 1.4644 - rightLayer2_loss: 0.9761 - val_loss: 5.3322 - val_leftLayer1_loss: 0.1012 - val_midLayer1_loss: 1.4886 - val_rightLayer1_loss: 1.1572 - val_leftLayer2_loss: 0.1014 - val_midLayer2_loss: 1.3547 - val_rightLayer2_loss: 1.1291
Epoch 5/11
1673/1690 [============================>.] - ETA: 0s - loss: 5.1889 - leftLayer1_loss: 0.0987 - midLayer1_loss: 1.4922 - rightLayer1_loss: 1.0995 - leftLayer2_loss: 0.0880 - midLayer2_loss: 1.4627 - rightLayer

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 1690 steps, validate for 200 steps
Epoch 1/11
1670/1690 [============================>.] - ETA: 0s - loss: 6.5012 - leftLayer1_loss: 0.1228 - midLayer1_loss: 1.4162 - rightLayer1_loss: 1.7406 - leftLayer2_loss: 0.1256 - midLayer2_loss: 1.5484 - rightLayer2_loss: 1.5475
Epoch 00001: val_loss improved from inf to 6.20180, saving model to ./experiments/0.013999999999999995_weights.h5
1690/1690 [==============================] - 5s 3ms/step - loss: 6.4974 - leftLayer1_loss: 0.1228 - midLayer1_loss: 1.4167 - rightLayer1_loss: 1.7389 - leftLayer2_loss: 0.1255 - midLayer2_loss: 1.5485 - rightLayer2_loss: 1.5448 - val_loss: 6.2018 - val_leftLayer1_loss: 0.1207 - val_midLayer1_loss: 1.4312 - val_rightLayer1_loss: 1.6092 - val_leftLayer2_loss: 0.1198 - val_midLayer2_loss: 1.4378 - val_rightLayer2_loss: 1.4

1674/1690 [============================>.] - ETA: 0s - loss: 4.9750 - leftLayer1_loss: 0.0850 - midLayer1_loss: 1.4162 - rightLayer1_loss: 0.9569 - leftLayer2_loss: 0.0619 - midLayer2_loss: 1.5502 - rightLayer2_loss: 0.9048
Epoch 00011: val_loss improved from 5.07967 to 5.05698, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Edema: 0.6980659229008188

auprc 0Edema: 0.06496844865004439

auroc 1Edema: 0.37464642532762427

auprc 1Edema: 0.013378423688262477

auroc 2Edema: 0.7472869150431152

auprc 2Edema: 0.06404868915775894

auroc 3Edema: 0.38733118472531025

auprc 3Edema: 0.013494781055211175

auroc 4Edema: 0.7735919690002265

auprc 4Edema: 0.06820152801346979

auroc 5Edema: 0.5361231699995381

auprc 5Edema: 0.018550855901443116

mean auroc: 0.5861742644994389

mean auprc: 0.04044045441103165

max auroc: 0.7735919690002265

max auprc: 0.0682015

Epoch 4/11
1684/1690 [============================>.] - ETA: 0s - loss: 5.2536 - leftLayer1_loss: 0.1125 - midLayer1_loss: 1.4147 - rightLayer1_loss: 1.2659 - leftLayer2_loss: 0.0981 - midLayer2_loss: 1.3668 - rightLayer2_loss: 0.9957
Epoch 00004: val_loss improved from 5.56187 to 5.39487, saving model to ./experiments/0.014999999999999994_weights.h5
1690/1690 [==============================] - 4s 3ms/step - loss: 5.2532 - leftLayer1_loss: 0.1124 - midLayer1_loss: 1.4146 - rightLayer1_loss: 1.2657 - leftLayer2_loss: 0.0980 - midLayer2_loss: 1.3669 - rightLayer2_loss: 0.9955 - val_loss: 5.3949 - val_leftLayer1_loss: 0.1111 - val_midLayer1_loss: 1.4178 - val_rightLayer1_loss: 1.2354 - val_leftLayer2_loss: 0.1019 - val_midLayer2_loss: 1.3569 - val_rightLayer2_loss: 1.1717
Epoch 5/11
1688/1690 [============================>.] - ETA: 0s - loss: 5.1225 - leftLayer1_loss: 0.1087 - midLayer1_loss: 1.4153 - rightLayer1_loss: 1.1756 - leftLayer2_loss: 0.0906 - midLayer2_loss: 1.3720 - rightLayer

In [17]:
print(np.max(maxi))

0.8162035173834925
